Included installs

In [1]:
# !pip install pandas

In [2]:
# !pip install influxdb

In [3]:
# !pip install influxdb-client

In [4]:
# !pip install matplotlib

In [5]:
# !pip install fuzzywuzzy

In [6]:
# !pip install openpyxl

Not included installs, but neessary

In [7]:
# !pip install beautifulsoup4

In [8]:
# !pip install feedparser

In [9]:
# !pip install lxml

In [10]:
import pandas as pd, numpy as np
import json
from influxdb_client import InfluxDBClient, Point, WritePrecision, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS

Set params

In [11]:
purge = False
# purge=True

In [12]:
process_annoations = False
# process_annoations=True

In [13]:
# process_macro=False
process_macro = True

In [14]:
# write_out=False
write_out = True

In [15]:
full_overwrite = False
# full_overwrite=True

Load pre-params

In [16]:
df0 = pd.read_csv("df0.csv").set_index("date")
df0.index = pd.to_datetime(df0.index)

In [17]:
bets = json.loads(open("bets.json", "r").read())

In [18]:
severity = pd.read_csv("severity.csv").set_index("date")
severity.index = pd.to_datetime(severity.index)

In [19]:
mobility = pd.read_csv("mobility.csv").set_index("date")
mobility.index = pd.to_datetime(mobility.index)

In [ ]:
mobility_mini = pd.read_csv("mobility_mini.csv").set_index("date")
mobility_mini.index = pd.to_datetime(mobility_mini.index)

Establish DB connection

In [21]:
user = "admin"
token = open("auth/influxb.txt", "r").read()
org = "roeim"
bucket = "base"
# write_api = client.write_api(write_options=SYNCHRONOUS)

In [22]:
if purge:
    client = InfluxDBClient(url="http://influxdb:8086", token=token)
    delete_api = client.delete_api()
    start = str(pd.to_datetime("1970-01-02")).replace(" ", "T") + "Z"
    stop = str(pd.to_datetime("now")).replace(" ", "T") + "Z"
    delete_api.delete(start, stop, "", bucket=bucket, org=org)
    client.close()

In [23]:
htmlipath = "../html/"
htmlepath = "//myserv.er/"
htmlepath_other = "//mybackupserv.er/"

In [24]:
titles = {"HU": "Magyar", "RO": "Română", "EN": "English"}

In [25]:
rtitles = {titles[t]: t for t in titles}

In [26]:
import requests

grafana = "http://grafana:3000/"
headers = {
    "Authorization": "Bearer " + open("auth/grafana.txt", "r").read(),
    "Accept": "application/json",
    "Content-Type": "application/json",
}

In [27]:
response = requests.get(grafana + "api/folders", headers=headers)
folders = json.loads(response.content)

In [28]:
folder_id = [f["id"] for f in folders if f["title"] == "My Grafana Folder"][0] #General

In [29]:
response = requests.get(grafana+'api/search?folderIds='+str(folder_id), headers=headers)
dashs = json.loads(response.content)

In [30]:
uids = {rtitles[d["title"]]: d["uid"] for d in dashs if d["title"] in rtitles}
iids = {rtitles[d["title"]]: d["id"] for d in dashs if d["title"] in rtitles}

In [31]:
uids_light = {
    rtitles[d["title"].split(" Light")[0]]: d["uid"]
    for d in dashs
    if "Light" in d["title"]
}
iids_light = {
    rtitles[d["title"].split(" Light")[0]]: d["id"]
    for d in dashs
    if "Light" in d["title"]
}

In [32]:
languages = ["HU", "RO", "EN"]

In [33]:
url='https://docs.google.com/spreadsheets/d/'+open('auth/sheet.txt','r').read()+'/gviz/tq?tqx=out:csv&sheet='

In [34]:
def get_szotar():
    sheet = "szotar"
    columns = (
        languages
        + [i + "_description" for i in languages]
        + [i + "_source" for i in languages]
    )
    df = pd.read_csv(url + sheet)
    df = df[["ID", "UI"] + columns]
    sheet = "minidashboard"
    df2 = pd.read_csv(url + sheet)
    df2 = df2[["ID", "UI"] + columns]
    df = pd.concat([df, df2])
    szotardf = df.set_index("ID")[columns]
    szotar = df.set_index("ID").T.to_dict()
    szotarHU = df.set_index("HU", drop=False).T.to_dict()
    szotarRO = df.set_index("RO", drop=False).T.to_dict()
    szotarEN = df.set_index("EN", drop=False).T.to_dict()
    return szotardf, szotar, szotarHU, szotarRO, szotarEN

In [35]:
def resolve_time_conflicts(series):
    ds = {}
    ts = []
    for d in series:
        if d not in ds:
            ds[d] = pd.to_datetime(d)
            t = ds[d]
        else:
            ds[d] = ds[d] + pd.to_timedelta("193m")
            t = ds[d]
        ts.append(t)
    return ts

# Influx

In [36]:
szotardf, szotar, szotarHU, szotarRO, szotarEN = get_szotar()

<ipython-input-34-77652dbc15c9>:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarHU=df.set_index('HU',drop=False).T.to_dict()
<ipython-input-34-77652dbc15c9>:13: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarRO=df.set_index('RO',drop=False).T.to_dict()
<ipython-input-34-77652dbc15c9>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarEN=df.set_index('EN',drop=False).T.to_dict()


In [37]:
import time
import datetime
import pytz

utc = pytz.UTC

In [38]:
def push2influx(
    df,
    measurement,
    field_columns,
    tag_columns,
    shift=False,
    wo=write_out,
    fo=full_overwrite,
    daily=True,
    bucket="base",
):
    if wo:
        client = InfluxDBClient(url="http://influxdb:8086", token=token)
        write_api = client.write_api(
            write_options=WriteOptions(
                batch_size=1000,
                flush_interval=10_000,
                jitter_interval=2_000,
                retry_interval=5_000,
            )
        )
        df = df.sort_index()
        # df.index = utc.localize(df.index)
        df.index = df.index.tz_localize("GMT")  # .tz_convert('Europe/Bucharest')
        if shift:
            df.index += pd.to_timedelta("12h")
        if fo:
            pass
            delete_api = client.delete_api()
            start = str(pd.to_datetime("1970-01-02")).replace(" ", "T") + "Z"
            stop = str(pd.to_datetime("now")).replace(" ", "T") + "Z"
            delete_api.delete(
                start,
                stop,
                '_measurement="' + measurement + '"',
                bucket=bucket,
                org=org,
            )
        else:
            query = """
                from(bucket: "base")
                |> range(start:-25h, stop: now())
                |> filter(fn: (r) => r._measurement == """+'"'+ measurement +'"'
            latest = client.query_api().query_data_frame(org=org, query=query)
            if "_time" in latest.columns:
                lat = latest["_time"][0]
                if daily:
                    lat += pd.to_timedelta("1d")
                df = df[lat:]
                print("Slicing", measurement, "from", lat, "...")
            else:
                print("No data in db for", measurement, "...")
        time.sleep(3)
        print("Writing to", measurement, "...")
        write_api.write(
            bucket,
            org=org,
            record=df[field_columns + tag_columns],
            data_frame_measurement_name=measurement,
            data_frame_tag_columns=tag_columns,
        )
        time.sleep(2)
        client.close()
        print("Done!")
    else:
        print("Write-out not enabled. Skipping...")

DateLaZi

In [39]:
# new method from DateLaZi
dlz = requests.get("https://datelazi.ro/latestData.json").content
dlz = json.loads(dlz)
dlz["historicalData"][dlz["currentDayStats"]["parsedOnString"]] = dlz["currentDayStats"]
dlzs = []
dlz_counties = []
for date in dlz["historicalData"]:
    d = {}
    mdate = (
        date.replace("2018-11-07", "2020-11-07")
        .replace("2020-01-05", "2021-01-05")
        .replace("2020-01-07", "2021-01-07")
    )
    d["date"] = mdate
    d["cases"] = dlz["historicalData"][date]["numberInfected"]
    d["heals"] = dlz["historicalData"][date]["numberCured"]
    d["deaths"] = dlz["historicalData"][date]["numberDeceased"]
    if "vaccines" in dlz["historicalData"][date]:
        if dlz["historicalData"][date]["vaccines"]:
            for v in dlz["historicalData"][date]["vaccines"]:
                for k in ["total_administered", "immunized"]:
                    k2 = k.replace("immunized", "total_immunized")
                    if k2 not in d:
                        d[k2] = 0
                    d[k2] += dlz["historicalData"][date]["vaccines"][v][k]
                    d[k2 + "_" + v] = dlz["historicalData"][date]["vaccines"][v][k]
    if "countyInfectionsNumbers" in dlz["historicalData"][date]:
        counties = dlz["historicalData"][date]["countyInfectionsNumbers"]
        if counties:
            for county in counties:
                dlz_counties.append(
                    {
                        "date": mdate,
                        county: dlz["historicalData"][date]["countyInfectionsNumbers"][
                            county
                        ],
                    }
                )
    dlzs.append(d)
dl = pd.DataFrame(dlzs).set_index("date").sort_index(ascending=True)

In [40]:
for c in dl.columns[3:]:
    dl[c] = dl[c].cumsum()
dl2 = dl.diff()
dl2.columns = ["case", "heal", "death"] + [
    i.replace("total_", "") for i in dl2.columns[3:]
]
dl["active"] = dl["cases"] - dl["heals"] - dl["deaths"]
dl = dl.join(dl2)

In [41]:
latest = dl.index[-1]
now = str(pd.to_datetime("now"))[:10]
print(latest, now)

2021-03-28 2021-03-29


Tests from `graphs.ro`

In [42]:
r = requests.get("https://www.graphs.ro/json.php")
graphs_ro = json.loads(r.content)["covid_romania"]
g = pd.DataFrame(graphs_ro)
# g['date']=pd.to_datetime(g['reporting_date'])
g["date"] = g["reporting_date"]
g = g.set_index("date")
g["tests"] = g["total_tests"]
g["test"] = g["new_tests_today"]
latest2 = g.index[0]
print(latest2)

2021-03-26


In [43]:
latest = str(min(pd.to_datetime(latest), pd.to_datetime(latest2)))[:10]
print(latest)

2021-03-26


In [44]:
dl = dl.join(g[["tests", "test"]])

Governance

In [45]:
sheet = "governance"

In [46]:
df = pd.read_csv(url + sheet)
df = df.set_index("date").sort_index(ascending=True).dropna(how="all")
df = df.join(dl, lsuffix="_g")  # [20:] #[:-1] vaccines nincs a governance-ban

In [47]:
# fill from local, if missing in DateLaZi
for c in ["active", "cases", "heals", "deaths", "case", "heal", "death"]:
    df[c] = (
        df[c]
        .fillna(df[c + "_g"].astype(str).str.replace(",", ""))
        .replace("nan", np.nan)
    )
df = df.dropna(how="all", axis=0).dropna(how="all", axis=1)

In [48]:
df14 = df[["cases"]].reset_index().join(df[["cases"]][14:].reset_index(), lsuffix="1")
df14["case14"] = df14["cases"].astype(float) - df14["cases1"].astype(float)
df = df.join(df14.set_index("date")["case14"].dropna().astype(int))

In [49]:
df.index = pd.to_datetime(df.index)
df0 = df.copy()
vaccine_totals = [
    i
    for i in list(dl2.columns[3:]) + list(dl.columns[3 : len(dl2.columns)])
    if "total_" in i
]
df = (
    pd.DataFrame(
        df[["active", "cases", "heals", "deaths", "case14"] + vaccine_totals].stack()
    )
    .reset_index()
    .set_index("date")
)
df.columns = ["type", "value"]
df["value"] = df["value"].astype(str).str.replace(",", "").astype(float).astype(int)
df = df.join(szotardf, on="type")
df = (
    pd.DataFrame(df.reset_index().set_index(["date", "type", "value"]).stack())
    .reset_index()
    .set_index("date")
)
df.columns = ["type", "value", "lang", "langtype"]
current = df[df["type"] == "case14"]["value"][-1]

In [50]:
df0.to_csv("df0.csv")

In [51]:
# df0=pd.read_csv('df0.csv').set_index('date')

-> Run till here to test Grafana only.

In [52]:
field_columns = ["value"]
tag_columns = ["type", "lang", "langtype"]
measurement = "governance1"
# push2influx(df.replace(0,np.nan).dropna(),measurement,field_columns,tag_columns,fo=True)
push2influx(df.replace(0, np.nan).dropna(), measurement, field_columns, tag_columns)

Writing to governance1 ...
Done!


In [53]:
df = df0.copy()

Daily cases

In [54]:
df["case/test"] = (
    100
    * df["case"].astype(str).str.replace(",", "").astype(float)
    / df["test"].astype(str).str.replace(",", "").astype(float)
)
df["death_rate"] = (
    100
    * df["death"].astype(str).str.replace(",", "").astype(float)
    / df["case"].astype(str).str.replace(",", "").astype(float)
)

In [55]:
df = (
    pd.DataFrame(df[["death", "heal", "case", "case/test", "death_rate"]].stack())
    .reset_index()
    .set_index("date")
)
df.columns = ["type", "value"]
df["value"] = df["value"].astype(str).str.replace(",", "").astype(float)
df = df.join(szotardf, on="type")
df = (
    pd.DataFrame(df.reset_index().set_index(["date", "type", "value"]).stack())
    .reset_index()
    .set_index("date")
)
df.columns = ["type", "value", "lang", "langtype"]

In [56]:
field_columns = ["value"]
tag_columns = ["type", "lang", "langtype"]
measurement = "governance2"
# push2influx(df.replace(0,np.nan).dropna(),measurement,field_columns,tag_columns,fo=True)
push2influx(df.replace(0, np.nan).dropna(), measurement, field_columns, tag_columns)

Writing to governance2 ...
Done!
